In [1]:

from pyspark import SparkContext
#importing sparkcontext

In [2]:
!pip install findspark

In [3]:
import findspark
findspark.init()

In [4]:
#Create a SparkContext
#1st arg --> where we want to run , 2nd arg --> Spark Application name
sc = SparkContext("local","SparkPractice")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 08:18:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
data = [x+1 for x in range(5)]

# paralleize helps to convert dataset to rdd
# There can be two arguments for parallelize function i.e. one argument for dataset and another
#one for no of partions
rdd = sc.parallelize(data)

In [6]:
result = rdd.collect()
print(result)

[1, 2, 3, 4, 5]


In [7]:
f = lambda x: print(x)
f(['a','b'])

['a', 'b']


In [8]:
names_rdd = sc.parallelize(['Alyal','Sagar','Brain','Mark'])
names_rdd.collect()

['Alyal', 'Sagar', 'Brain', 'Mark']

In [9]:
num = sc.parallelize([x for x in range(7)],4)

In [10]:
num.getNumPartitions()

4

In [11]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.app.submitTime', '1695370721316'),
 ('spark.app.id', 'local-1695370727534'),
 ('spark.executor.id', 'driver'),
 ('spark.app.startTime', '1695370722027'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.re

In [12]:

num.countByValue()

defaultdict(int, {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1})

In [13]:
partitioned_data = num.glom().collect()

#Display the contents of each partitions
for i,partition in enumerate(partitioned_data):
    print(f"Partition{i+1}:{partition}")

Partition1:[0]
Partition2:[1, 2]
Partition3:[3, 4]
Partition4:[5, 6]


In [14]:
a = sc.parallelize([1,2,3,4,5]).foreach(lambda x:print(x))

1
2
3
4
5


In [15]:
data = [('Alice',25),('Bob',30),('Charlie',35),('David',28),('Eve',22)]

In [16]:
from pyspark.sql import SparkSession

In [17]:
from pyspark.sql.types import StructType , StructField , IntegerType ,StringType

In [18]:
schema = StructType([
    StructField("name",StringType(),True),
    StructField("age",IntegerType(),True)
])

In [19]:
spk = SparkSession.builder.appName("Alyal").getOrCreate()

In [20]:
rdd = spk.sparkContext.parallelize(data)

In [21]:
rdd.collect()

[('Alice', 25), ('Bob', 30), ('Charlie', 35), ('David', 28), ('Eve', 22)]

In [22]:
df = spk.createDataFrame(rdd,schema)

In [23]:
df.show()

+-------+---+
|   name|age|
+-------+---+
|  Alice| 25|
|    Bob| 30|
|Charlie| 35|
|  David| 28|
|    Eve| 22|
+-------+---+



In [24]:
df[df['age']>30].show()

+-------+---+
|   name|age|
+-------+---+
|Charlie| 35|
+-------+---+



In [25]:
df.filter(df.age >= 30).show()

+-------+---+
|   name|age|
+-------+---+
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [26]:
df.createOrReplaceTempView('mytable') #Create a temporary view

In [27]:
result = spk.sql("SELECT * FROM mytable WHERE age>=30")

In [28]:
result.show()

+-------+---+
|   name|age|
+-------+---+
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [29]:
movie_df = spk.read.csv('movies.csv',header=True,inferSchema=True)

In [30]:
movie_df.show()

+--------------------+---------+--------------------+----------------+-------------+-----------------+---------------+----+
|                Film|    Genre|         Lead Studio|Audience score %|Profitability|Rotten Tomatoes %|Worldwide Gross|Year|
+--------------------+---------+--------------------+----------------+-------------+-----------------+---------------+----+
|Zack and Miri Mak...|  Romance|The Weinstein Com...|              70|  1.747541667|               64|        $41.94 |2008|
|     Youth in Revolt|   Comedy|The Weinstein Com...|              52|         1.09|               68|        $19.62 |2010|
|You Will Meet a T...|   Comedy|         Independent|              35|  1.211818182|               43|        $26.66 |2010|
|        When in Rome|   Comedy|              Disney|              44|          0.0|               15|        $43.04 |2010|
|What Happens in V...|   Comedy|                 Fox|              72|  6.267647029|               28|       $219.37 |2008|
| Water 

In [31]:
movie_df.createOrReplaceTempView('mytable') #Create a temporary view

In [32]:
movie_df.columns

['Film',
 'Genre',
 'Lead Studio',
 'Audience score %',
 'Profitability',
 'Rotten Tomatoes %',
 'Worldwide Gross',
 'Year']

In [33]:
movie_df.summary().show()

23/09/22 08:19:14 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+-------+----------------+-----------------+------------------+------------------+---------------+------------------+
|summary|                Film|  Genre|     Lead Studio| Audience score %|     Profitability| Rotten Tomatoes %|Worldwide Gross|              Year|
+-------+--------------------+-------+----------------+-----------------+------------------+------------------+---------------+------------------+
|  count|                  77|     77|              77|               77|                77|                77|             77|                77|
|   mean|                null|   null|            null|63.72727272727273|4.5994833979610386|46.701298701298704|           null| 2009.077922077922|
| stddev|                null|   null|            null|13.65711339967137| 8.031990152409822| 26.09500100093026|           null|1.3549736234200407|
|    min|(500) Days of Summer| Action|20th Century Fox|               35|               0.0|                 3|       

In [34]:
movie_df.dtypes

[('Film', 'string'),
 ('Genre', 'string'),
 ('Lead Studio', 'string'),
 ('Audience score %', 'int'),
 ('Profitability', 'double'),
 ('Rotten Tomatoes %', 'int'),
 ('Worldwide Gross', 'string'),
 ('Year', 'int')]

In [35]:
print(movie_df.schema)

StructType([StructField('Film', StringType(), True), StructField('Genre', StringType(), True), StructField('Lead Studio', StringType(), True), StructField('Audience score %', IntegerType(), True), StructField('Profitability', DoubleType(), True), StructField('Rotten Tomatoes %', IntegerType(), True), StructField('Worldwide Gross', StringType(), True), StructField('Year', IntegerType(), True)])


In [36]:
result = spk.sql('SELECT Film,Genre FROM MYTABLE WHERE Year > 2007')
result.show()

+--------------------+---------+
|                Film|    Genre|
+--------------------+---------+
|Zack and Miri Mak...|  Romance|
|     Youth in Revolt|   Comedy|
|You Will Meet a T...|   Comedy|
|        When in Rome|   Comedy|
|What Happens in V...|   Comedy|
| Water For Elephants|    Drama|
|              WALL-E|Animation|
| Waiting For Forever|  Romance|
|     Valentine's Day|   Comedy|
|Twilight: Breakin...|  Romance|
|            Twilight|  Romance|
|      The Ugly Truth|   Comedy|
|The Twilight Saga...|    Drama|
|The Time Traveler...|    Drama|
|        The Proposal|   Comedy|
|The Invention of ...|   Comedy|
|         The Duchess|    Drama|
|The Curious Case ...|  Fantasy|
|    The Back-up Plan|   Comedy|
|             Tangled|Animation|
+--------------------+---------+
only showing top 20 rows



In [37]:

movie_df.select("Film", "Genre","Audience score %","Rotten Tomatoes %").summary("count","mean","stddev").show()

+-------+----+-----+-----------------+------------------+
|summary|Film|Genre| Audience score %| Rotten Tomatoes %|
+-------+----+-----+-----------------+------------------+
|  count|  77|   77|               77|                77|
|   mean|null| null|63.72727272727273|46.701298701298704|
| stddev|null| null|13.65711339967137| 26.09500100093026|
+-------+----+-----+-----------------+------------------+



In [40]:
sc.stop()

In [51]:
from pyspark import SparkConf , SparkContext
conf = SparkConf().setAppName('App1').setMaster('local')
sc1 = SparkContext(conf=conf)
num = sc1.parallelize([5,5,4,3,2,9,2])
num.collect()

[5, 5, 4, 3, 2, 9, 2]

In [52]:
num2 = num.map(lambda x:x*2)

In [53]:
num2.collect()

[10, 10, 8, 6, 4, 18, 4]

In [49]:
list(map(lambda x:x*2,[5,5,4,3,2,9,2]))

[10, 10, 8, 6, 4, 18, 4]

In [55]:
names = sc1.parallelize(['A','B','C','D'])

In [56]:
names_mr = names.map(lambda x:'Mr.'+x)
names_mr.collect()

['Mr.A', 'Mr.B', 'Mr.C', 'Mr.D']

In [59]:
rdd = sc1.parallelize([2,3,7])
rdd.flatMap(lambda x: range(1,x)).collect()

[1, 1, 2, 1, 2, 3, 4, 5, 6]

In [67]:
rdd.map(lambda x: range(1,x)).collect()

[range(1, 2), range(1, 3), range(1, 7)]

In [65]:
def generate_squares(number):
    return [(number,number**2)]

In [66]:
numbers = sc1.parallelize([1,2,3,4,5])
squares_rdd = numbers.flatMap(generate_squares)
squares_rdd.collect()

[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]

In [88]:
squares_rdd.filter(lambda x:x if x[1]>10 else 0 ).collect()

[(4, 16), (5, 25)]

In [99]:
even_result = sc1.parallelize([2,4,6,8,10])
odd_result = sc1.parallelize([1,3,5,7,9])
x= even_result.union(odd_result)

In [100]:
even_result.union(odd_result).glom().collect()

[[2, 4, 6, 8, 10], [1, 3, 5, 7, 9]]

In [101]:
# The fold method is used to add values to 
x.fold(4,lambda a,b:a+b)

67